In [3]:
import numpy as np
import pandas as pd
import time

from datetime import datetime, timedelta

## Structure of the algorithm

The following code is divided in two classes namely, 

- ConsumptionModel
- Simulator

Consumption model primarily consists of the functions related to the basic machine learning model which predicts the dynamic user behaviour

In [ ]:
class ConsumptionModel(object):
    def __init__(self):
#         some variables
    
    def train(self, X, y, hyperparameters):
#         complete the xgboost model on 2013 data
    
    def test(self, X, y):
#         test the data points

    def entropy(self, y):
#         get entropy of each data point nad return the entropy dataframe




    

In [ ]:
class Simulator:
    
    def __init__(self):
        self.datetime = {"season":, "DoW":, "special_event":, "tariff_policy":}
        self.df = pd.DataFrame()
        self.user_engagement = *randomly select users of specific percentage*
        self.participation = *randomly select users of specific percentage*
        self.user_latency = *select random variable for smoothing function*

    def select_day(self, datetime):
#         add the provided variables and call _datetime function also include the randomness
#         return the selected datetime details

        
    def auto_noise_addition(self):
#         select the random users and their behaviour with random latency


    def tariff_policy(self, levels, constraints):
#         use variables from auto_noise_addition and input variables of this function to create a tariff policy 
#         for each participating user **REQUIRES TIME AND BRAIN**

        
    def run(self, df_n):
#         FOR EACH USER, call test function of consumption model, get modified behaviour, return original data point and modified data point
        

In [ ]:
# Make a dictionary similar to but not limited to following dictionary
# selection_vars = {"season":"1-to-4", "DoW":"1-to-7", "special_day":"select holiday/event", "tariff_policy":"24-hour-policy-list"}

class select_next(object):
    
    def __init__(self):
        self.datetime = {"season":, "DoW":, "special_event":, "tariff_policy":}
    
    
    def random_day(self):
#         select the variables randomly, put them in self.datetime dictionary and call _datetime function 

    
    
    def specific_day(self, season, dow, event):
#         add the provided variables and call _datetime function also include the randomness
        
    def select_day(self, datetime):
#         add the provided variables and call _datetime function also include the randomness
#         return the selected datetime details
        
    def tariff_policy(self, levels, constraints):
        
        
    